<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/CV19_Synthesis_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/covid"
INPUT_DIR = '/input_mix' #mean, std
dataset_class = "training"
source_file =PATH+ INPUT_DIR+'/validation/cleaned/cleaned_worldwide_cases_27_04.csv'
TARGET_DIR = PATH+  INPUT_DIR+'/'+dataset_class+'/cleaned/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
original_data = pd.read_csv(source_file)
original_data.head()

,cases,deaths,popData2018,countriesAndTerritories,geoId,countryterritoryCode,continentExp,_country_code,synthesis_seq
0,1,0,14439018,1,1,1,1,1,0
1,1,0,14439018,1,1,1,1,1,0
2,0,0,14439018,1,1,1,1,1,0
3,0,1,14439018,1,1,1,1,1,0
4,0,0,14439018,1,1,1,1,1,0


In [0]:
def generate_synthetic_data(sample_dataset, window_mean, window_std, sythesize_ratio = 2, forced_reverse = False):
    synthetic_data = pd.DataFrame(columns=sample_dataset.columns)
    synthetic_data.insert(len(sample_dataset.columns), "synthesis_seq", [], True) 

    
    for k in range(sythesize_ratio):
      if len(synthetic_data) >= len(sample_dataset) * sythesize_ratio:
        break;
      #this loop generates a set that resembles the entire dataset
      for country in sample_dataset['countryterritoryCode'].unique():
        country_synthetic = pd.DataFrame(columns=synthetic_data.columns)
        input_sequence_len = int(np.random.normal(window_mean, window_std))  #needs to be randomized

        #population data change
        country_data_i = sample_dataset[sample_dataset['countryterritoryCode'] == country]
        if len(country_data_i) < input_sequence_len :
          continue
        feature_length = 2 #number of features to be randomized
        country_data_array = np.array([country_data_i['cases'].to_numpy(), country_data_i['deaths'].to_numpy()])
        country_data_array = country_data_array.reshape(feature_length,len(country_data_i))
        x = country_data_array[:feature_length].T
        
        reversed = np.random.normal(0,1)>0
        if reversed:
          x = x[::-1]
            
        sets =0
        x_list = []
        dict_x = dict()
        for i in range(input_sequence_len):
          array_len = ((len(x) -i) - ((len(x)-i)%input_sequence_len))+i
          if array_len <= 0:
            continue
          sets = int( array_len/ input_sequence_len)
          if sets <= 0:
            continue
          #print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
          x_temp = x[i:array_len].T.reshape(sets,feature_length,input_sequence_len)
          uniq_keys = np.array([i+(input_sequence_len*k) for k in range(sets)])
          x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
          arrays_split = np.hsplit(x_temp,sets)
          dict_x.update(dict(zip(uniq_keys, arrays_split)))

        temp_x_list  = [dict_x[i].T for i in sorted(dict_x.keys())]        
        temp_x_list = np.array(temp_x_list).squeeze()
        feature_means = np.mean(temp_x_list, axis=1)
        feature_std = np.std(temp_x_list, axis=1)
        random_normal_feature_values = np.random.normal(feature_means, feature_std).T
        random_normal_feature_values = np.round(random_normal_feature_values,0)
        random_normal_feature_values[random_normal_feature_values < 0] = 0

        if reversed:
          random_normal_feature_values = random_normal_feature_values.T[::-1]
          random_normal_feature_values = random_normal_feature_values.T

        country_synthetic['cases'] = pd.Series(random_normal_feature_values[0])
        country_synthetic['deaths'] = pd.Series(random_normal_feature_values[1])

        
        popData2018 = country_data_i['popData2018'].values[0]
        population_change_percentage = 1+ (np.random.normal(0,1) / 100)
        population_new = np.round(population_change_percentage*popData2018)

        country_synthetic['popData2018'] = population_new
        country_synthetic['_country_code'] = country_data_i['_country_code'].values[0]
        country_synthetic['countriesAndTerritories'] = country_data_i['countriesAndTerritories'].values[0]
        country_synthetic['countryterritoryCode'] = country_data_i['countryterritoryCode'].values[0]
        country_synthetic['geoId'] = country_data_i['geoId'].values[0]
        country_synthetic['continentExp'] = country_data_i['continentExp'].values[0]
        country_synthetic['synthesis_seq'] = k
        synthetic_data = synthetic_data.append(country_synthetic, ignore_index=True)
    return synthetic_data

In [0]:
run_id = 552022 #ddmyyhh
for i in range(10):
  window_mean = i+6
  window_std= 1
  synthetic_data  = generate_synthetic_data(original_data, window_mean = window_mean,  window_std= window_std, sythesize_ratio = 14)
  synthetic_data.to_csv(TARGET_DIR + '/synthetic_run_'+str(i)+'_'+str(run_id)+'_dt_27_04'+'_wmean_'+str(window_std)+'_wstd_'+str(window_std)+'.csv',  index=False )
  print('synth step : ', i, ' len : ', len(synthetic_data))
print(os.path.listdir(TARGET_DIR))

synth step :  0  len :  170597
synth step :  1  len :  167837
synth step :  2  len :  164898
synth step :  3  len :  162192


In [0]:
#for development purpose. To generate data, there is a cell below.
for i in range(5):
  synthetic_data = generate_synthetic_data(original_data, window_mean = 10, window_std = 2, sythesize_ratio = 1, forced_reverse= True)

first_country  = [c for c in  original_data['countryterritoryCode'].unique()][3]
plt1, = plt.plot(synthetic_data[synthetic_data['countryterritoryCode'] == first_country]['cases'].tolist())
plt2, = plt.plot(original_data[original_data['countryterritoryCode'] == first_country]['cases'].tolist())
plt.xlabel('number of days')
plt.ylabel('confirmed case count')
plt.title(first_country)

plt.legend([plt1,plt2], ['Synthetic '+ str(len(synthetic_data[synthetic_data['countryterritoryCode'] == first_country]['cases']))
                         ,'Real ' + str(len(original_data[original_data['countryterritoryCode'] == first_country]['cases']))])

In [0]:
#testing for accuracy
def verify_synthetic_data(synthetic_data,original_data, print_results = False):
  countryterritoryCode ='CN'
  #test_synthetic_data = synthetic_data[synthetic_data['countryterritoryCode'] == countryterritoryCode]
  test_synthetic_data = synthetic_data
  control_set = original_data #[original_data['countryterritoryCode']==countryterritoryCode]
  case_mean = np.mean(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy())
  death_mean = np.mean(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy())
  pop_mean = np.mean(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy())

  case_std = np.std(control_set['cases'].to_numpy()) - np.mean(test_synthetic_data['cases'].to_numpy())
  death_std = np.std(control_set['deaths'].to_numpy()) - np.mean(test_synthetic_data['deaths'].to_numpy())
  pop_std = np.std(control_set['popData2018'].to_numpy()) - np.mean(test_synthetic_data['popData2018'].to_numpy())

  if print_results:
    print(test_synthetic_data.head())
    print(control_set.head())
    print('original data length : ', len(control_set))
    print('synthetic data length : ',len(test_synthetic_data))
    print('min sequence length : ',min([len(test_synthetic_data[test_synthetic_data['countryterritoryCode']==country]) for country in test_synthetic_data['countryterritoryCode'].unique()]))
    print('max sequence length : ',max([len(test_synthetic_data[test_synthetic_data['countryterritoryCode']==country]) for country in test_synthetic_data['countryterritoryCode'].unique()]))

    print('Mean diff. cases : ', case_mean)
    print('Mean diff. deaths : ', death_mean)
    print('Mean diff. population : ', pop_mean)
  
  return  [case_mean, death_mean, pop_mean, case_std , death_std , pop_std ]

In [0]:
"""
#for development
results = []
for i in range(5,15):
   synthetic_data = generate_synthetic_data(original_data, window_mean = i, window_std = 1, sythesize_ratio = 1)
   results.append(verify_synthetic_data(synthetic_data,original_data))
   plt.plot(np.array(results).T[0])
plt.plot(np.array(results).T[1])
"""

In [0]:
run_id = 2842001 #ddmyyhh
